In [1]:
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objs as go
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import collections
import spacy

In [2]:
# loading joblisting dataset from github
url_markets = 'https://raw.githubusercontent.com/johnsiphiwe/EDSA-Project-2022/Scrapping-Joas/CSV_files/Global_Markets.csv'
url_global = 'https://raw.githubusercontent.com/johnsiphiwe/EDSA-Project-2022/Scrapping-Joas/CSV_files/indeed_jobs%20abroad.csv'
url_local_jobs = 'https://raw.githubusercontent.com/johnsiphiwe/EDSA-Project-2022/Scrapping-Joas/CSV_files/local_jobs.csv.csv'
job_listing = pd.concat(
    map(pd.read_csv, [url_markets, url_local_jobs, url_local_jobs]))


job_listing.head()

,Unnamed: 0,title,company,location,job_desc,date_posted,date_scrapped,salary,job_url,Unnamed: 0.1
0,0,Lead Data Scientist (Remote),Boclips,London,Are you ready to shape the future of media-ric...,None,2022-05-19,"£90,000 per year",https://www.careerjet.co.za/jobad/gb514a18cd3f...,NaN
1,1,Head of Data Science| Remote | £80K - £110k | ...,Opus Recruitment Solutions,South East England,"Hampshire, United Kingdom Permanent £80000 ...",18 days ago,2022-05-19,"£80,000-110,000 per year",https://www.careerjet.co.za/jobad/gb4dbe27d5a0...,NaN
2,2,Cost Engineer (Life Sciences/Pharma/Data Centres),Turner & Townsend,Newcastle,Company Description At Turner & Townsend we’...,4 days ago,2022-05-19,NaN,https://www.careerjet.co.za/jobad/gb257e728bea...,NaN
3,3,Project Controls Engineer - Planning (Life Sci...,Turner & Townsend,Manchester,Company Description At Turner & Townsend we’...,4 days ago,2022-05-19,NaN,https://www.careerjet.co.za/jobad/gbb1b7fb4de1...,NaN
4,4,Project Controls Engineer - Planning (Life Sci...,Turner & Townsend,Newcastle,Company Description At Turner & Townsend we’...,4 days ago,2022-05-19,NaN,https://www.careerjet.co.za/jobad/gbf3e2a6e657...,NaN


In [3]:
# remove duplicates
job_listing.drop_duplicates()


,Unnamed: 0,title,company,location,job_desc,date_posted,date_scrapped,salary,job_url,Unnamed: 0.1
0,0,Lead Data Scientist (Remote),Boclips,London,Are you ready to shape the future of media-ric...,None,2022-05-19,"£90,000 per year",https://www.careerjet.co.za/jobad/gb514a18cd3f...,NaN
1,1,Head of Data Science| Remote | £80K - £110k | ...,Opus Recruitment Solutions,South East England,"Hampshire, United Kingdom Permanent £80000 ...",18 days ago,2022-05-19,"£80,000-110,000 per year",https://www.careerjet.co.za/jobad/gb4dbe27d5a0...,NaN
2,2,Cost Engineer (Life Sciences/Pharma/Data Centres),Turner & Townsend,Newcastle,Company Description At Turner & Townsend we’...,4 days ago,2022-05-19,NaN,https://www.careerjet.co.za/jobad/gb257e728bea...,NaN
3,3,Project Controls Engineer - Planning (Life Sci...,Turner & Townsend,Manchester,Company Description At Turner & Townsend we’...,4 days ago,2022-05-19,NaN,https://www.careerjet.co.za/jobad/gbb1b7fb4de1...,NaN
4,4,Project Controls Engineer - Planning (Life Sci...,Turner & Townsend,Newcastle,Company Description At Turner & Townsend we’...,4 days ago,2022-05-19,NaN,https://www.careerjet.co.za/jobad/gbf3e2a6e657...,NaN
...,...,...,...,...,...,...,...,...,...,...
6310,2890,C# Software Developer / Analyst,NaN,"Cape Town, Western Cape",2+ years’ experience within the software devel...,Posted30+ days ago,2022-05-09,NaN,https://www.indeed.com/rc/clk?jk=4c8147b884822...,1510.0
6311,2891,Credit Optimisation Senior Analyst,"<a class=""turnstileLink companyOverviewLink"" d...","Parow, Western Cape","Excellent data analysis, analytical and proble...",Posted30+ days ago,2022-05-09,NaN,https://www.indeed.com/rc/clk?jk=4c8147b884822...,1511.0
6312,2892,Mid-Senior Technical Test Analyst,NaN,South Africa,Manage and maintain the test environments and ...,Posted30+ days ago,2022-05-09,NaN,https://www.indeed.com/rc/clk?jk=4c8147b884822...,1512.0
6313,2893,Software Engineer,NaN,"Stellenbosch, Western Cape",Proficient in writing fine-tuned SQL queries a...,Posted30+ days ago,2022-05-09,NaN,https://www.indeed.com/rc/clk?jk=4c8147b884822...,1513.0


In [4]:
def experience(x):

    junior = ['internship', 'junior', 'graduate']
    mid_level = ['3 years', '4 years', 'exceptional', 'experience']
    senior = ['sr', 'manager', '6 year', '10 years', 'lead']

    for word in x.lower().split():
        if word in junior:
            return 'Junior'
        elif word in senior:
            return 'Senior'
        elif word in mid_level:
            return 'Mid_level 3-5 years'
        else:
            return 'Mid_level 1-2 years'


# apply the experience feature
job_listing['experience'] = job_listing['job_desc'].apply(experience)


In [5]:
# province feature
def province(x):
    type_prov = []
    #x= x.str.replace('[^\w\s\d]',' ')
    provinces = ['gauteng', 'johannesburg', 'sandton', 'parktown', 'natal', 'limpopo', 'durban', 'kwazulu', 'midrand', 'stellenbosch', 'centurion',
                 'western', 'cape', 'pretoria', 'town', 'midrand' 'limpopo', 'north', 'west', 'eastern Cape', 'free' 'State', 'mpumalanga', 'northen cape']
    remote = []
    for word in x.lower().split():
        if word in provinces:
            type_prov.append(word)
        elif word not in provinces:
            remote.append(word)
    for y in type_prov:
        if y == 'johannesburg' or y == 'gauteng' or y == 'midrand' or y == 'pretoria' or y == 'centurion':
            return 'Gauteng'
        elif y == 'stellenbosch' or y == 'western' or y == 'cape':
            return 'Western Cape'
        elif y == 'durban' or y == 'kwazulu':
            return 'Kwazulu-Natal'
        elif y == 'limpopo':
            return 'Limpopo'
        elif y == 'eastern':
            return 'Eastern Cape'
        elif y == 'free state':
            return 'Free State'
        elif y == 'limpopo':
            return 'Limpopo'
        elif y == 'northen cape':
            return 'Northen Cape'
        elif y == 'mpumalanga':
            return ' Mpumalanga'
        elif y == 'north' or 'west':
            return 'North West'
    for z in remote:
        if z in str(['united states', 'somerville', 'plano', 'dearborn', 'MI', 'sydney', 'NSW', 'Tampa', 'FL', 'Dallas', 'TX', 'San', 'Francisco', 'CA', 'Brooklyn', 'NY', 'US', 'Santa Clara', 'Seattle', 'WA', 'Plano', 'TX', 'New York City', 'NY', 'San Diego', 'Mountain View', 'CA', 'Livonia', 'MI', 'McLean', 'IL', 'Winter Park', 'FL']).lower():
            return 'Remote in the US'
        if z in str(['England', 'London', 'Cambridge', 'Manchester', 'Newcastle', 'Strand', 'Bristol Area', 'Bristol Area', 'Oxford', 'Birmingham', 'Nottingham', 'Liverpool', 'County Durham', 'England', 'Cambridge', 'Berkshire', 'Greater London']).lower():
            return 'Remote in England'
        if z in str(['AUE', 'Dubai,Abu Dhabi']).lower():
            return 'Remote in AUE'
        if z in str(['Autralia', 'Canberra', 'ACT']).lower():
            return 'Remote in Australia'
        if z in ['singapore']:
            return 'Remote in Singapore'
        else:
            return 'Remote'
        
# apply function
job_listing['province'] = job_listing['location'].apply(province)



In [18]:
# country feature
def country(x):
    provinces = ['Gauteng', 'Limpopo', 'Western Cape', 'Free State',
                 'Eastern Cape', 'Kwazulu-Natal', 'Mpumalanga', 'North West', 'Northern Cape']
    if x in provinces:
        return 'South Africa'
    elif x == 'Remote in the US':
        return 'United States of America'
    elif x == 'Remote in England':
        return 'England'
    elif x == 'Remote in AUE':
        return 'UAE'
    elif x == 'Remote in Singapore':
        return 'Singapore'
    elif x == 'Remote':
        return 'Remote'


# apply feature
job_listing['country'] = job_listing['province'].apply(country)


In [19]:
# creating numbers days from date posted
job_listing['number_days'] = job_listing['date_posted'].str.extract(
    '(\d+)', expand=False)
# change date scrapped to datetime
job_listing['date_scrapped'] = pd.to_datetime(job_listing['date_scrapped'])
# convert number days to date and minus
job_listing['number_days'] = job_listing['number_days'].fillna(0).astype(int)
# creating new feature date
job_listing['date'] = (job_listing['date_scrapped'] -
                       pd.to_timedelta(job_listing['number_days'], unit='d'))

In [20]:
def fix_title(x):

    machine_learning = ['machine', 'learning']
    data_scientist = ['data', 'scientist', 'science']
    bus_intel = ['business', 'intelligence', 'developer']
    data_analyst = ['data', 'analyst', ]
    aws = ['aws', 'data', 'science', 'architect']
    data_engineer = ['engineer', 'engineering']
    software = ['software']

    for word in x.lower().split():
        if word in machine_learning:
            return 'Machine Learning'
        elif word in data_scientist:
            return 'Data scientist'
        elif word in bus_intel:
            return 'Business Intelligence'
        elif word in aws:
            return 'AWS Data Science Architect'
        elif word in data_analyst:
            return 'Data analyst'
        elif word in data_engineer:
            return 'Data engineer'
        elif word in software:
            return 'Software Developer'
        else:
            return 'Data Science'


# applying to function to create new feature
job_listing['summary_title'] = job_listing['job_desc'].apply(fix_title)


## Visualize

In [21]:
job_listing_2 = pd.DataFrame(job_listing.groupby(['country', 'province', 'date', 'summary_title', 'job_url']).size(), columns=['count']).reset_index()
job_listing_2.set_index('date', inplace = True)
job_listing_2.sort_values(by='date', ascending = False, inplace=True)
job_listing_2

,country,province,summary_title,job_url,count
date,,,,,
2022-05-19,United States of America,Remote in the US,Data Science,https://www.careerjet.co.za/jobad/use2f40b3399...,22
2022-05-19,England,Remote in England,Data Science,https://www.careerjet.co.za/jobad/gb09d1cfbc7e...,15
2022-05-19,England,Remote in England,Data Science,https://www.careerjet.co.za/jobad/gb26265b3f00...,15
2022-05-19,England,Remote in England,Data Science,https://www.careerjet.co.za/jobad/gb514a18cd3f...,21
2022-05-19,England,Remote in England,Data Science,https://www.careerjet.co.za/jobad/gb5a0cbe1c76...,15
...,...,...,...,...,...
2022-04-09,South Africa,Gauteng,Data Science,https://www.indeed.com/company/Jenrec-Pty-Ltd/...,8
2022-04-09,South Africa,Gauteng,Data Science,https://www.indeed.com/company/Kazang-Connect/...,6
2022-04-09,South Africa,Gauteng,Data Science,https://www.indeed.com/company/Virtual-School-...,12


In [22]:
unstacked_job = job_listing_2.pivot_table('count', ['date', 'summary_title', 'job_url'], 'country')
unstacked_job = round(unstacked_job.fillna(0),0)

In [23]:
unstacked_job.sort_values(by='date', ascending = False, inplace=True)
unstacked_job.reset_index(inplace = True)
unstacked_job.rename(index={'': 'Row_1'}, inplace=True)

In [24]:
unstacked_job

country,date,summary_title,job_url,England,Remote,South Africa,United States of America
0,2022-05-19,Machine Learning,https://www.careerjet.co.za/jobad/gb1241b2fea6...,21.0,0.0,0.0,0.0
1,2022-05-19,Data Science,https://www.careerjet.co.za/jobad/us49de481fc7...,0.0,0.0,0.0,22.0
2,2022-05-19,Data Science,https://www.careerjet.co.za/jobad/gb09d1cfbc7e...,15.0,0.0,0.0,0.0
3,2022-05-19,Data Science,https://www.careerjet.co.za/jobad/gb17c10fc37f...,15.0,0.0,0.0,0.0
4,2022-05-19,Data Science,https://www.careerjet.co.za/jobad/gb26265b3f00...,15.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
1056,2022-04-09,Data Science,https://www.indeed.com/rc/clk?jk=bc780fc842205...,0.0,2.0,6.0,0.0
1057,2022-04-09,Data Science,https://www.indeed.com/rc/clk?jk=c469ac55fc495...,0.0,8.0,14.0,0.0
1058,2022-04-09,Data Science,https://www.indeed.com/rc/clk?jk=c926b289ad960...,0.0,4.0,12.0,0.0
1059,2022-04-09,Data Science,https://www.indeed.com/rc/clk?jk=cec496d186610...,0.0,6.0,8.0,0.0


In [25]:
unstacked_job2= unstacked_job.copy()

In [30]:
trace1 = go.Scatter(x= unstacked_job['date'],
                y= unstacked_job['England'],
               marker ={'color': '#FFD700'})

trace2 = go.Scatter(x= unstacked_job['date'],
                y= unstacked_job['Remote'],
                marker ={'color': 'green'})

trace3 = go.Scatter(x= unstacked_job['date'],
                 y= unstacked_job['South Africa'],
                marker ={'color': 'orange'})

#trace4 = go.Bar(x= unstacked_job['summary_title'],
                 #y= unstacked_job['Singapore'],
               #marker ={'color': 'red'})

#trace5 = go.Bar(x= unstacked_job['date'],
                #y= unstacked_job['UAE'],
                #marker ={'color': 'blue'})

trace6 = go.Scatter(x= unstacked_job['date'],
                y= unstacked_job['United States of America'],
                marker ={'color': 'white'})

data = [trace1, trace2,trace3, trace6]


layout = go.Layout(title = 'count of jobs', barmode = 'stack')
fig= go.Figure(data=data,layout=layout)
pyo.plot(fig)

'temp-plot.html'

### skills required for a Data Science job

In [5]:
nltk.download('stopwords') 
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sjtsi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#remove stopswords
removed_stop_words = set(stopwords + list(string.punctuation) + list(string.digits))

In [42]:
#job_listing[job_listing['job_desc'] != '-1']['job_desc'].tolist()

In [7]:
#Check the most used words in job title

job_desc = job_listing[job_listing['job_desc'] != '-1']['job_desc'].tolist()
list_of_words = []
for item in job_desc:
    content = nltk.word_tokenize(item)
    for word in content:
        # remove unwanted words
        if word.lower() not in removed_stop_words and word.lower() != 'data'and word.lower() != 'analyst' and word.lower() != 'data analyst' and 'science'and word.lower() != 'scientist' and word.lower() != 'machine learning' and word.lower() != 'engineer' and word.lower() != 'sciemtist':
            list_of_words.append(word.upper())

In [20]:
list_of_words1 = pd.DataFrame({'job_desc': list_of_words}) 
list_of_words1

,job_desc
0,READY
1,SHAPE
2,FUTURE
3,MEDIA-RICH
4,LEARNING
...,...
220271,ORGANISATION
220272,SETTING
220273,ENTERPRISE
220274,REPORTING


In [21]:
list_of_words1['job_desc'].unique()


array(['READY', 'SHAPE', 'FUTURE', ..., 'CONSCIOUSLY', 'STRIVE',
       'INCREASE'], dtype=object)

In [23]:
attributes = list_of_words1['job_desc'].value_counts().sort_values(ascending = False).nlargest(n=20).reset_index()
attributes

,index,job_desc
0,SCIENCE,3895
1,TEAM,2798
2,…,2413
3,JOIN,2014
4,BUSINESS,1816
5,CLIENT,1800
6,EXPERIENCE,1672
7,COMPANY,1657
8,FINANCIAL,1609
9,ANALYSIS,1582


### The most listed skills using job description

To extract name entity recognition from all job titles and save in a dictionary

In [24]:
nlp = spacy.load('en_core_web_sm')

name_entity_recognition_job_desc = {}

for item in job_desc:
    token = nlp(item)
    for ent in token.ents:      
        name_entity_recognition_job_desc.setdefault(ent.label_ , []).append(ent.text) 

In [25]:
name_entity_recognition_job_desc.keys()


dict_keys(['ORG', 'GPE', 'MONEY', 'PERSON', 'LOC', 'NORP', 'DATE', 'CARDINAL', 'FAC', 'PERCENT', 'PRODUCT', 'WORK_OF_ART', 'QUANTITY', 'ORDINAL', 'TIME', 'LAW'])

In [27]:
collections.Counter(name_entity_recognition_job_desc['ORG']).most_common()


[('Deloitte', 408),
 ('SAS', 358),
 ('SQL', 340),
 ('Data Analyst', 329),
 ('Data Science Pipeline', 302),
 ('Data Science', 262),
 ('Bank’s', 252),
 ('the Best Bank', 248),
 ('A Data Science Team Lead', 204),
 ('Company & Team', 204),
 ('IHS Markit', 204),
 ('Computational/ Applied Mathematics/ Statistics', 204),
 ('PowerBI', 204),
 ('Stakeholder', 204),
 ('Industrial Engineer', 174),
 ('Data Extraction and Analysis', 152),
 ('R / Scala', 148),
 ('Extract, Transform', 148),
 ('Turner & Townsend', 147),
 ('Anglo American / De Beers Group', 136),
 ('Contribute', 128),
 ('AI', 90),
 ('On-Demand and Logistics', 88),
 ('UK Curriculum', 88),
 ('UK Curriculum | International GCSE', 88),
 ('Graduate', 86),
 ('Bibliographic', 74),
 ('Requisition Details & Talent Acquisition Contact', 72),
 ('CallMiner/Speech', 72),
 ('EU', 63),
 ('Data Quality', 60),
 ('Digital Data Analyst', 60),
 ('BI Tools – Design Dashboards/Charts/Custom Reports', 60),
 ('Inventory', 60),
 ('+ Private Health Insurance', 5

In [30]:
collections.Counter(name_entity_recognition_job_desc['DATE']).most_common()


[('3 years', 86),
 ('At least two years', 52),
 ('10 May 2022', 36),
 ('May 2022', 36),
 ('1 - 2 years', 36),
 ('today', 30),
 ('28/04/2022', 30),
 ('daily', 28),
 ('11 West 19th', 22),
 ('22008', 22),
 ('At least 1 year', 22),
 ('at least 30 month', 21),
 ('2330', 21),
 ('12 month', 21),
 ('May 11 2022', 21),
 ("3-5 years'", 20),
 ('5 years', 16),
 ('6 Month', 15),
 ('5+ years', 12),
 ('4+ years', 12),
 ('2 years', 10),
 ('monthly', 10),
 ('At least 1 years', 10),
 ('3+ years', 6),
 ('2-3 years', 6),
 ('At least 3 years', 6),
 ('19050', 4),
 ('1+ year’s', 4),
 ('2 to 3 years', 4),
 ('quarterly', 4),
 ('2+ years', 4),
 ('More than 5 years', 4),
 ('Monthly', 4),
 ("5 + years'", 4),
 ('1-5 years', 4),
 ('documentation.1 - 2 years', 4),
 ('Intraday', 4),
 ("7 years'", 4),
 ('1st', 2),
 ('1-2 years', 2),
 ('yearly', 2),
 ('At least one year', 2),
 ('3-4 years', 2),
 ('skills.2 - 4 years', 2),
 ('114804', 2),
 ('5 + years', 2),
 ('4 to 5 years', 2),
 ('3 -5 years', 2),
 ('2-5 years', 2),
 (

In [31]:
#As seen above, the job title column lists a few practical technological skills.

skills = ['SAS','POWERBI', 'AWS', 'SPARK', 'SQL', 'AZURE', 'PYTHON', 'EXCEL', 'R / Scala']


In [36]:
count_skills_word = []
for word in list_of_words:
    if word in skills:
        count_skills_word.append(word)
        
list_of_words2 = pd.DataFrame({'skills': count_skills_word}) 
skills = list_of_words2.value_counts().sort_values(ascending = False).nlargest(n=20).reset_index()


In [37]:
skills

,skills,0
0,AWS,1291
1,PYTHON,552
2,SQL,407
3,SAS,360
4,POWERBI,266
5,EXCEL,128
6,SPARK,63
7,AZURE,4
